**Mô tả bài tập** 
    
    Đi đến web https://www.bestprice.vn/ve-may-bay, dùng selenium điền các thông tin địa điểm đi là Hà nội, địa điểm đế Hồ Chí Minh, chọn ngày đi là ngày mai sau đó tìm tất cả chuyến bay có thể sau đó cào thông tin các chuyến bay đó về .

    Lưu ý: trang web có thể bị thay đổi cấu trúc làm code có thể không chạy được sau này ạ :(

In [179]:
# import các thư viện cần thiết
from datetime import date, timedelta
import time
import numpy as np
import json
import re
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

In [180]:
#Load trang web 
driver = webdriver.Chrome(executable_path='.\Driver\chromedriver.exe')
driver.get("https://www.bestprice.vn/ve-may-bay")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15312\157375108.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='.\Driver\chromedriver.exe')


In [181]:
# Chờ cho quảng cáo xuất hiện và tắt tránh ảnh hưởng đến các thao tác phía sau 
time.sleep(20)

In [182]:
#Tắt quảng cáo
driver.find_element(By.XPATH, "//div[@class='modal-content fancybox-opened']//button[@class='close fancybox-close']//span").click()

In [183]:
#Chọn địa điểm đi (Hà nội)
driver.find_element(By.XPATH, "//label[@class='btn_flight_icon']").click()
driver.find_element(By.XPATH, "//li[@class='col-xs-6 col-sm-3 li-item-des des-chd flight_from_2']").click()

#Chọn địa điểm đến (Hồ Chí Minh)
driver.find_element(By.XPATH, "//label[@for='To']").click()
driver.find_element(By.XPATH, "//li[@class='col-xs-6 col-sm-3 li-item-des des-chd flight_to_3']").click()


In [184]:
# # Click chỗ ngày đi để hiện lên calendar để chọn
driver.find_element(By.XPATH, "//div[@class='search-form__content__date__input']").click()

#  Tính ngày mai là ngày mấy
today = date.today()
tomorrow = today + timedelta(1)
tomorrow_day = str(tomorrow.day)
tomorow_month = tomorrow.strftime('%b')

#Chọn ngày đi là ngày mai
dayy = driver.find_elements(By.XPATH, "//td[@class=' undefined']//span[1]")
for i in dayy:
    if i.text == tomorrow_day:
        i.click()
        break

# Ấn nút tìm kiếm để tìm chuyến bay
driver.find_element(By.XPATH, "//button[@id='search_button']").click()

In [185]:
#Chờ cho trang web load các chuyển bay tìm được 
time.sleep(10)


In [186]:
# Craw mã chuyến bay lưu vào data
df = pd.DataFrame()

MCB = driver.find_elements(By.XPATH, "//div[@class='margin-left-3 list-flight-code']//span[@class='flight-code']")
i = 0
col = []
for i in MCB:
    col.append(i.text)
df['MCB'] = col

In [187]:
# Craw thời gian đi của chuyến bay lưu vào data
TG_FROM = driver.find_elements(By.XPATH, "//div[@class=' col-xs-4 col-sm-3 col-time-fly text-center']//span[1][@class='flight-code']")

i = 0
col = []
for i in TG_FROM:
    col.append(i.text)
df['TG_FROM'] = col

In [188]:
# Craw thời gian đến của chuyến bay lưu vào data
TG_TO = driver.find_elements(By.XPATH, "//div[@class=' col-xs-4 col-sm-3 col-time-fly text-center']//span[3][@class='flight-code']")

i = 0
col = []
for i in TG_TO:
    col.append(i.text)
df['TG_TO'] = col

In [189]:
# Craw chi phí của chuyến bay lưu vào data
PHI = driver.find_elements(By.XPATH, "//label//b//span")
PHI
i = 0
col = []
for i in PHI:
    col.append(i.text)
df['PHI'] = col


In [190]:
# Click vào ô chi tiết của mỗi chuyến bay để cào thêm thông tin 
tt = driver.find_elements(By.XPATH, "//div[@class='col-xs-4 col-sm-3 text-center col-detail hidden-xs']//span[@class='show-detail']//span[@type='button']")

for i in tt:
    i.click()
    time.sleep(1/5) # chờ mỗi lần click tránh mạng íu bị miss 1 số lần click


In [191]:
# Craw thuế của chuyến bay lưu vào data
THUE = driver.find_elements(By.XPATH, "//div[@id='data_price_fee']//span")

i = 0
col = []
for i in THUE:
    col.append(i.text)
df['THUE'] = col

In [192]:
# Craw thuế của chuyến bay lưu vào data
THUE = driver.find_elements(By.XPATH, "//div[@id='data_price_fee']//span")

i = 0
col = []
for i in THUE:
    col.append(i.text)
df['THUE'] = col

In [193]:
# Craw tổng tiền của chuyến bay lưu vào data
SUM_TIEN = driver.find_elements(By.XPATH, "//div[@class='text-right div-total-price']//span[@id='data_price_total']//span")

i = 0
col = []
for i in SUM_TIEN:
    col.append(i.text)
df['SUM_TIEN'] = col

In [194]:
# Craw loại máy bay của chuyến bay lưu vào data( TH không có thông tin loại MB để trống)
LOAI_MB = driver.find_elements(By.XPATH, "//div[@class='row ruler-itinerary-flight']//div[2][@class='col-xs-6 pd-right-0']")

i = 0
col = []
for i in LOAI_MB:
    element = i.text
    tmp = element.split("\n")
    if len(tmp) > 2: 
        col.append(tmp[-1])
    else:
        col.append('')

df['LOAI_MB'] = col



In [195]:
# Data các chuyến bay 
df

,MCB,TG_FROM,TG_TO,PHI,THUE,SUM_TIEN,LOAI_MB
0,VJ-157,23:10,01:20,699.000đ,718.000đ,1.417.000đ,
1,VU-781,18:15,20:30,1.028.000đ,754.000đ,1.782.000đ,
2,VN-6009,20:10,22:25,1.069.000đ,776.000đ,1.845.000đ,Airbus A320
3,VN-223,23:00,01:15,1.219.000đ,768.000đ,1.987.000đ,Airbus A321
4,QH-203,07:20,09:40,1.269.000đ,792.000đ,2.061.000đ,Boeing 787
5,VJ-139,14:20,16:30,938.000đ,747.000đ,1.685.000đ,Airbus A321
6,VJ-147,18:25,20:35,938.000đ,747.000đ,1.685.000đ,Airbus A321
7,VJ-149,19:40,21:50,938.000đ,747.000đ,1.685.000đ,
8,VJ-151,18:55,21:05,938.000đ,747.000đ,1.685.000đ,Airbus A320
9,VJ-159,15:05,17:10,938.000đ,747.000đ,1.685.000đ,Airbus A330
